In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn import datasets
from sklearn import model_selection

import math

In [ ]:
train_file = '../input/happy-whale-and-dolphin/train.csv'
df = pd.read_csv(train_file)
df.species.replace({"globis": "short_finned_pilot_whale",
                      "pilot_whale": "short_finned_pilot_whale",
                      "kiler_whale": "killer_whale",
                      "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)
df.to_csv('train_fixed.csv',index=False)

In [ ]:
df.head()

In [ ]:
species_set = set(df['species'].values)
print("Now, we have " + str(len(species_set)) + " classes of whale and dolphin: ")

In [ ]:
id_set = set(df['individual_id'].values)
print("Now, we have " + str(len(id_set)) + " whales and dolphins: ")

In [ ]:
for species in species_set:
    print(species)

In [ ]:

def create_folds(data, num_splits):
    data["kfold"] = -1
    data["code"] = -1
    cat = 0
    for species in species_set:
        for i in range(len(data)):
            if data["species"][i] == species:                
                data["code"][i] = cat
        cat = cat + 1
    # print(cat)
    # print(data)
    num_bins = int(np.floor(1 + math.log(len(data),len(species_set))))
    
    data.loc[:, "bins"] = pd.cut(data["code"], bins=num_bins, labels=False)

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)
    data = data.drop("code", axis=1)
    return data


df = pd.read_csv("./train_fixed.csv")

df_5 = create_folds(df, num_splits=5)
df_5.to_csv("train_5folds.csv", index=False)
df_10 = create_folds(df, num_splits=10)
df_10.to_csv("train_10folds.csv", index=False)